In [1]:
# !pip install yfinance tqdm pytz --quiet

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

# ============================================
# 1) 종목 리스트
# ============================================
symbols = ["SPY"]

# ============================================
# 2) 시계열 데이터 수집
# ============================================
START = "2022-11-07"
END   = "2025-11-09"

print("[1/2] 시계열 데이터 수집 중 ...")
data = yf.download(symbols, start=START, end=END, interval="1d", auto_adjust=False)

# UTC → 뉴욕시간 변환
data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

# 전날 가격을 다음날 개장시간(09:30)에 반영
session_dates = (data.index + pd.Timedelta(days=1)).normalize()
session_times = session_dates + pd.Timedelta(hours=9, minutes=30)
data.index = session_times

print("NY 개장 시간으로 라벨 조정 완료")
print("기간:", data.index.min().date(), "~", data.index.max().date())

# ============================================
# 3) MultiIndex → 단일 컬럼으로 변환
# ============================================
data.columns = [f"{col}_{sym}" for col, sym in data.columns]

# ============================================
# 4) 필요한 컬럼만 선택
# ============================================
ohlcv = data[["Close_SPY"]].copy()

# ============================================
# 5) Date 컬럼 생성
# ============================================
ohlcv["Date"] = ohlcv.index.tz_localize(None).normalize().date
ohlcv.reset_index(drop=True, inplace=True)

# Date 컬럼을 맨 앞으로 이동
ohlcv = ohlcv[["Date"] + [col for col in ohlcv.columns if col != "Date"]]

# ============================================
# 6) 출력
# ============================================
print(ohlcv.head(10))

[1/2] 시계열 데이터 수집 중 ...


[*********************100%***********************]  1 of 1 completed

NY 개장 시간으로 라벨 조정 완료
기간: 2022-11-07 ~ 2025-11-07
         Date   Close_SPY
0  2022-11-07  379.950012
1  2022-11-08  382.000000
2  2022-11-09  374.130005
3  2022-11-10  394.690002
4  2022-11-11  398.510010
5  2022-11-14  395.119995
6  2022-11-15  398.489990
7  2022-11-16  395.450012
8  2022-11-17  394.239990
9  2022-11-18  396.029999


In [4]:
ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       754 non-null    object 
 1   Close_SPY  754 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.9+ KB


In [5]:
ohlcv

,Date,Close_SPY
0,2022-11-07,379.950012
1,2022-11-08,382.000000
2,2022-11-09,374.130005
3,2022-11-10,394.690002
4,2022-11-11,398.510010
...,...,...
749,2025-11-03,683.340027
750,2025-11-04,675.239990
751,2025-11-05,677.580017
752,2025-11-06,670.309998


In [6]:
print(ohlcv.columns)

Index(['Date', 'Close_SPY'], dtype='object')


In [7]:
ohlcv.to_csv("../output_data/top60_y_target_data.csv", index=False)